# 개별 csv 저장

In [ ]:
# 각각의 종목별 차트 데이터 수집


api_key = ''
api_secret = ''

from datetime import datetime
import pandas as pd

COLUMNS = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 
                   'tb_base_av', 'tb_quote_av', 'ignore']


from binance.client import Client
client = Client(api_key=api_key, api_secret=api_secret)

def get_data(start_date, end_date, symbol):
    data = []
    
    data = client.futures_historical_klines(
            symbol=symbol,
            interval='1m',
            start_str=start_date,
            end_str=end_date
        )
    
    # 전처리
    df = pd.DataFrame(data)
    df.columns = COLUMNS
    df['open_time'] = df.apply(lambda x:datetime.fromtimestamp(x['open_time'] // 1000), axis=1)
    df = df.drop(columns = ['close_time', 'ignore'])
    df['symbol'] = symbol
    df.loc[:, 'open':'tb_quote_av'] = df.loc[:, 'open':'tb_quote_av'].astype(float)  # string to float
    df['trades'] = df['trades'].astype(int)
    return df

start_date = '2024-01-14'
end_date = '2024-01-15'
symbols = [
    'BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'TRBUSDT', 'LINKUSDT', 'BNBUSDT',
    'AVAXUSDT', 'MATICUSDT', 'XRPUSDT', 'ETCUSDT', 'FILUSDT', 'DOGEUSDT',
    'NEARUSDT', 'ADAUSDT', 'LTCUSDT', 'BCHUSDT', 'FTMUSDT', 'ATOMUSDT',
    'EOSUSDT', 'RUNEUSDT', 'CRVUSDT', 'UNIUSDT', 'TRXUSDT', 'MKRUSDT',
    'XTZUSDT', 'EGLDUSDT', 'ALGOUSDT', 'XMRUSDT', 'NEOUSDT', 'COMPUSDT',
    'THETAUSDT', 'SNXUSDT', 'XLMUSDT', 'ZILUSDT'
]
for i in symbols:
    df = get_data(start_date=start_date, end_date=end_date, symbol=i)
    df.to_csv(f'D:\csv\hist_coin_data\{i}.csv', index=False)